In [1]:
# Importa las bibliotecas necesarias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
original = pd.read_csv('coches-de-segunda-mano-sample.csv')
df = original.copy()

In [3]:
df.drop(columns = ['url','company','doors','color', 'photos','is_professional', 'dealer', 'province', 'country', 'publish_date','insert_date'], inplace = True)

In [4]:
df['price_financed'] = df['price_financed'].fillna(0)

In [5]:
df.dropna(inplace = True)

In [6]:
numericas=['price','price_financed','kms','power']
def outliers(var):
    q3=var.quantile(0.75)
    q1=var.quantile(0.25)
    riq=q3-q1
    sup=q3+1.5*(riq)
    inf=q1-1.5*(riq)
    outl=(var>sup) | (var<inf)
    return outl

for i in df.loc[:,numericas]:
    print("Cantidad de oultliers de la variable",i,":",outliers(df[i]).sum())

for i in df.loc[:,numericas]:
    df=df[~outliers(df[i])]
df.sample(5)

Cantidad de oultliers de la variable price : 2040
Cantidad de oultliers de la variable price_financed : 1081
Cantidad de oultliers de la variable kms : 521
Cantidad de oultliers de la variable power : 3271


,make,model,version,price,price_financed,fuel,year,kms,power,shift
33451,SEAT,Ibiza,SEAT Ibiza 1.0 TGI 66kW 90CV FR 5p.,15650,13650.0,Gas natural (CNG),2020.0,6532,90.0,Manual
6033,TOYOTA,Prius,TOYOTA Prius 1.8 HSD ADVANCE 5p.,8990,8990.0,Híbrido,2011.0,89000,136.0,Automático
23966,MERCEDES-BENZ,Viano,MERCEDES-BENZ Viano 2.2 CDI Fun Compacta 4p.,19695,0.0,Diésel,2010.0,95000,150.0,Automático
41959,FORD,Fiesta,FORD Fiesta 1.4 Beat 3p.,4450,0.0,Gasolina,2011.0,119000,96.0,Manual
16618,SEAT,Arona,SEAT Arona 1.0 TGI 66kW 90CV FR Edition 5p.,16740,14740.0,Gas natural (CNG),2019.0,14000,90.0,Manual


In [7]:
#Instancio el OneHot
onehot=OneHotEncoder()

onehot.fit(df[["make","model",'fuel','shift','version','kms']])
a=onehot.transform(df[["make","model",'fuel','shift','version','kms']])

encoded_df = pd.DataFrame(a.toarray(), columns=onehot.get_feature_names_out(["make","model",'fuel','shift','version','kms']))

df.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

df=pd.concat([df,encoded_df],axis=1)

df.drop(columns=["make","model",'fuel','shift','version','kms'],inplace=True)

print(df.shape)
df.sample(5)

(35963, 27263)


,price,price_financed,year,power,make_ABARTH,make_ALFA ROMEO,make_AUDI,make_AUSTIN,make_BMW,make_CADILLAC,...,kms_322200,kms_322900,kms_322954,kms_323000,kms_323530,kms_324000,kms_324200,kms_324252,kms_325000,kms_325306
21130,19700,0.0,2015.0,136.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3241,29105,26338.0,2021.0,136.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5721,13950,13950.0,2016.0,145.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35304,4500,0.0,2008.0,143.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13345,26000,22900.0,2021.0,141.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Divide los datos en conjuntos de entrenamiento y prueba
X = df.drop(columns = ['price'], inplace = False)
y = df['price']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
# Crea y entrena el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba
predicciones = modelo.predict(X_test)

# Evalúa el rendimiento del modelo
error_cuadratico_medio = mean_squared_error(y_test, predicciones)
print(f'Error Cuadrático Medio: {error_cuadratico_medio}')

Error Cuadrático Medio: 7911590.047898717


In [11]:
# Realiza predicciones en los nuevos datos
prediccion_10_anios = modelo.predict(df)

# Imprime la predicción
print(f'Predicción del precio en 10 años: {prediccion_10_anios}')

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- price
